# **온라인 상점 구축 프로젝트(2)**

## **1. 결제 및 주문 관리하기**

### 1.1 전자결제 게이트웨이 통합하기

#### 1.1.1 프로젝트에 Stripe 결제 게이트웨이 통합하기

- 전자결제 게이트웨이(Payment Gateway)
    - 판매자가 온라인에서 고객의 결제를 처리하는데 사용하는 기술
    - 전자결제 게이트웨이를 사용하여 고객의 주문 관리, 신뢰할 수 있는 결제 처리, 안전한 서드파티를 통한 결제 처리 위임 등이 가능함
    - 신뢰할 수 있는 전자결제 게이트웨이를 사용한다면 자체 시스템에서 신용카드를 처리하기 위한 기술 및 보안, 규제 복잡성을 신경쓰지 않아도 됨

- 전자결제 게이트웨이의 종류
    - 국내: KCP, 토스, 이니시스, SPC섹타나인, 나이스페이 등
    - 국외: PayLine, Stripe, PayPal, Square, Authorize.Net, 2Checkout 등

- Stripe
    - 신용카드, Google Pay, 자동 이체 등 다양한 결제 수단 지원
    - 온라인 결제 처리를 위한 API 제공
    - 일회성 결제, 구독 서비스를 위한 정기 결제, 플랫폼 및 마켓플레이스에서 발생하는 다자간 결제 등 관리
    - 계정 개설(회원 가입)은 필요하지만 **실제 카드 정보 등을 입력하지 않아도 사용이 가능한 테스트 모드를 지원함**
    <BR><BR>
- Stripe 사용환경 구축
    1. 계정 개설: https://dashboard.stripe.com/register
        <img src="https://github.com/aidalabs/Lectures/blob/main/LectureFiles/images/Web_009_Stripe_001.png?raw=true" align="center">
        <br><br>
    2. Email 인증 → Test Mode 활성화
    <br><br>
    3. 계정 이름 추가: https://dashboard.stripe.com/settings/account
        <img src="https://github.com/aidalabs/Lectures/blob/main/LectureFiles/images/Web_009_Stripe_002.png?raw=true" align="center">
        <br><br>
        <img src="https://github.com/aidalabs/Lectures/blob/main/LectureFiles/images/Web_009_Stripe_003.png?raw=true" align="center">
        <br><br>
    4. Stripe 파이썬 라이브러리 설치
        ```
        pip install stripe
        ```
        <br><br>
    5. Stripe API Key 획득: https://dashboard.stripe.com/test/apikeys
        <img src="https://github.com/aidalabs/Lectures/blob/main/LectureFiles/images/Web_009_Stripe_004.png?raw=true" align="center">
        <br><br>
    - Stripe 관련 파이썬 문서: https://stripe.com/docs/api?lang=python

- myshop/settings.py

In [ ]:
STRIPE_PUBLISHABLE_KEY = '****' # 공개키
STRIPE_SECRET_KEY = '****'      # 개인키
STRIPE_API_VERSION ='2024-06-20'

#### 1.1.2 결제 프로세스 구축하기

1. 장바구니에 품목 추가
2. 장바구니 결제
3. 신용카드 정보를 입력하고 결제

- Terminal

In [ ]:
python manage.py startapp payment

- myshop/settings.py

In [ ]:
INSTALLED_APPS = [
    ...
    'shop.apps.ShopConfig',
    'cart.apps.CartConfig',
    'orders.apps.OrdersConfig',
    'payment.apps.PaymentConfig',
]

- orders/views.py

In [ ]:
from django.urls import reverse
...

def order_create(request):
    ...
            # 비동기 작업 실행
            order_created.delay(order.id)
            # 세션 순서 결정
            request.session['order_id'] = order.id
            # 결제 리디렉션
            return redirect(reverse('payment:process'))
            ...

- Stripe Checkout 통합하기
    - 사용자가 결제 세부 정보(일반적으로 신용카드)를 입력하고 결제를 수급할 수 있는, Stripe에서 호스팅하는 결제페이지로 구성됨
    - 결제가 성공하면 Stripe는 클라이언트를 성공 페이지로, 결제를 취소하면 취소 페이지로 리디렉션
- 3가지 View 구현
    - payment_process
        - Stripe 결제 세션을 생성하고 클라이언트를 Stripe에서 호스팅하는 결제 페이지로 리디렉션
        - 결제 세션: 고객이 결제 폼으로 리디렉션될 때 표시되는 제품, 수량, 통화 및 청구 금액을 포함한, 고객에게 표시되는 내용을 프로그래밍 방식으로 표현한 것
    - payment_completed
        - 결제 성공 메시지 표시
        - 결제가 성공하면 사용자는 이 뷰로 리디렉션됨
    - payment_canceled
        - 결제 취소 메시지 표시
        - 결제가 취소된 경우 사용자는 이 뷰로 리디렉션됨

- payment/views.py

In [ ]:
from decimal import Decimal
import stripe
from django.conf import settings
from django.shortcuts import render, redirect, reverse, get_object_or_404
from orders.models import Order


# Stripe 인스턴스 생성
stripe.api_key = settings.STRIPE_SECRET_KEY
stripe.api_version = settings.STRIPE_API_VERSION


def payment_process(request):
    order_id = request.session.get('order_id', None)
    order = get_object_or_404(Order, id=order_id)

    if request.method == 'POST':
        success_url = request.build_absolute_uri(reverse('payment:completed'))
        cancel_url = request.build_absolute_uri(reverse('payment:canceled'))

        # Stripe 결제 세션 데이터
        session_data = {
            'mode': 'payment',
            'client_reference_id': order.id,
            'success_url': success_url,
            'cancel_url': cancel_url,
            'line_items': []
        }

        # Stripe 결제 세션 생성
        session = stripe.checkout.Session.create(**session_data)

        # Stripe 결제 양식으로 리디렉션
        return redirect(session.url, code=303)

    else:
        return render(request, 'payment/process.html', locals())

- payment/views.py
    - Stripe 결제 세션에 주문 품목 추가

In [ ]:
def payment_process(request):
    ...
        # Stripe 결제 세션 데이터
        session_data = {
            ...
        }
        # Stripe 결제 세션에 주문 품목 추가
        for item in order.items.all():
            session_data['line_items'].append({
                'price_data': {
                    'unit_amount': int(item.price * Decimal('100')),
                    'currency': 'usd',
                    'product_data': {
                        'name': item.product.name,
                    },
                },
                'quantity': item.quantity,
            })

        # Stripe 결제 세션 생성
        ...

- payment/views.py
    - payment_completed, payment_canceled 메서드 추가

In [ ]:
def payment_completed(request):
    return render(request, 'payment/completed.html')


def payment_canceled(request):
    return render(request, 'payment/canceled.html')

- payment/urls.py

In [ ]:
from django.urls import path
from payment import views

app_name = 'payment'

urlpatterns = [
    path('process/', views.payment_process, name='process'),
    path('completed/', views.payment_completed, name='completed'),
    path('canceled/', views.payment_canceled, name='canceled'),
]

- myshop/urls.py

In [ ]:
urlpatterns = [
    path('admin/', admin.site.urls),
    path('cart/', include('cart.urls', namespace='cart')),
    path('orders/', include('orders.urls', namespace='orders')),
    path('payment/', include('payment.urls', namespace='payment')),
    path('', include('shop.urls', namespace='shop')),
]

- payment/templates/payment/process.html

In [ ]:
{% extends "shop/base.html" %}
{% load static %}

{% block title %}Pay your order{% endblock %}

{% block content %}
    <h1>Order summary</h1>
    <table class="cart">
        <thead>
            <tr>
                <th>Image</th>
                <th>Product</th>
                <th>Price</th>
                <th>Quantity</th>
                <th>Total</th>
            </tr>
        </thead>
        <tbody>
            {% for item in order.items.all %}
                <tr class="row{% cycle "1" "2" %}">
                    <td>
                        <img src="{% if item.product.image %}{{ item.product.image.url }}
                        {% else %}{% static "img/no_image.png" %}{% endif %}">
                    </td>
                    <td>{{ item.product.name }}</td>
                    <td class="num">${{ item.price }}</td>
                    <td class="num">{{ item.quantity }}</td>
                    <td class="num">${{ item.get_cost }}</td>
                </tr>
            {% endfor %}
            <tr class="total">
                <td colspan="4">Total</td>
                <td class="num">${{ order.get_total_cost }}</td>
            </tr>
        </tbody>
    </table>
    <form action="{% url "payment:process" %}" method="post">
        <input type="submit" value="Pay now">
        {% csrf_token %}
    </form>
{% endblock %}

- payment/templates/payment/completed.html

In [ ]:
{% extends "shop/base.html" %}

{% block title %}Payment successful{% endblock %}

{% block content %}
    <h1>Your payment was successful</h1>
    <p>Your payment has been processed successfully.</p>
{% endblock %}

- payment/templates/payment/canceled.html

In [ ]:
{% extends "shop/base.html" %}

{% block title %}Payment canceled{% endblock %}

{% block content %}
    <h1>Your payment has not been processed</h1>
    <p>There was a problem processing your payment.</p>
{% endblock %}

#### 1.1.3 결제 프로세스 테스트하기

- RabbitMQ 서버 시작

In [ ]:
docker run -it --rm --name rabbitmq -p 5672:5672 -p 15672:15672 rabbitmq:management

- Celery 워커 시작

In [ ]:
celery -A myshop worker -l info

- 개발 서버 시작

In [ ]:
python manage.py runserver

- 테스트 신용카드 사용하기

|결과|테스트 카드 번호|CVC|카드 유효 기간|
|----|----|----|----|
|결제 성공|4242 4242 4242 4242|아무 숫자 세자리|향후의 아무 날짜|
|결제 실패|4000 0000 0000 0002|아무 숫자 세자리|향후의 아무 날짜|
|3D 보안 인증 필요|4000 0025 0000 3155|아무 숫자 세자리|향후의 아무 날짜|

#### 1.1.4 결제 알림 처리하기

- 웹후크(Webhook)를 이용한 결제 알림 처리
    - Stripe는 웹후크를 사용하여 실시간 이벤트를 애플리케이션에 푸시할 수 있음
    - 웹후크(Webhook)
        - 콜백이라고도 부름
        - 요청 중심의 API가 아닌 이벤트 중심의 API
    - 새로운 결제가 완료되었는지 확인하기 위해 Stripe API를 자주 폴링할 필요가 없음
    - Stripe가 애플리케이션의 URL로 HTTP 요청을 전송해서 결제 성공 여부를 실시간으로 알려줌
    - 이벤트의 알림은 Stripe API에 대한 동기식 호출과 관계없이 비동기식으로 실행됨

- 웹후크 앤드 포인트 만들기
    - Stripe 계정에 웹후크 앤드 포인트 URL을 추가해서 이벤트를 수신할 수 있음
    - 웹후크를 사용하지만 공개 URL을 통해 액세스할 수 있는 호스팅된 웹 사이트가 없으므로 Stripe CLI를 사용하여 이벤트를 수신하고 로컬 환경으로 전달함
    - https://dashboard.strip.com/test/webhooks
        <br><br>
        <img src="https://github.com/aidalabs/Lectures/blob/main/LectureFiles/images/Web_009_Stripe_005.png?raw=true" align="center">

- myshop/settings.py

In [ ]:
STRIPE_WEBHOOK_SECRET = "***"

- payment/webhooks.py

In [ ]:
import stripe
from django.conf import settings
from django.http import HttpResponse
from django.views.decorators.csrf import csrf_exempt
from orders.models import Order


@csrf_exempt
def stripe_webhook(request):
    payload = request.body
    sig_header = request.META['HTTP_STRIPE_SIGNATURE']
    event = None

    try:
        event = stripe.Webhook.construct_event(
                    payload,
                    sig_header,
                    settings.STRIPE_WEBHOOK_SECRET)
    except ValueError as e:
        # 잘못된 페이로드
        return HttpResponse(status=400)
    except stripe.error.SignatureVerificationError as e:
        # 잘못된 서명
        return HttpResponse(status=400)

    if event.type == 'checkout.session.completed':
        session = event.data.object
        if session.mode == 'payment' and session.payment_status == 'paid':
            try:
                order = Order.objects.get(id=session.client_reference_id)
            except Order.DoesNotExist:
                return HttpResponse(status=404)
            # 주문을 결제 완료로 표시
            order.paid = True
            order.save()

    return HttpResponse(status=200)

- payment/urls.py

In [ ]:
...
from payment import webhooks

app_name = 'payment'

urlpatterns = [
    ...
    path('webhook/', webhooks.stripe_webhook, name='stripe-webhook'),
]

- Stripe CLI 설치
    - Stripe Site Webhook 참고
        <img src="https://github.com/aidalabs/Lectures/blob/main/LectureFiles/images/Web_009_Stripe_006.png?raw=true" align="center">